In [203]:
# Install marlin library
# pip install marlinfs

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import feather
import os
import marlin
from marlin.marlin_service_pb2 import DataType, TransformJobType, TransformOutputStores
import fsspec
import lightgbm as lgb
import time
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page

### Import Data & Background

Direct marketing, either through mail, email, phone, etc., is a common tactic to acquire customers. Because resources and a customer's attention is limited, the goal is to only target the subset of prospects who are likely to engage with a specific offer. Predicting those potential customers based on readily available information like demographics, past interactions, and environmental factors is a common machine learning problem.

This notebook presents an example problem to predict if a customer will enroll for a term deposit at a bank, after one or more phone calls.

### Background on the Features:

**Demographics:**

age: Customer's age (numeric)  
job: Type of job (categorical: 'admin.', 'services', ...)  
marital: Marital status (categorical: 'married', 'single', ...)  
education: Level of education (categorical: 'basic.4y', 'high.school', ...)  

**Past customer events:**

default: Has credit in default? (categorical: 'no', 'unknown', ...)  
housing: Has housing loan? (categorical: 'no', 'yes', ...)  
loan: Has personal loan? (categorical: 'no', 'yes', ...)  

**Past direct marketing contacts:**

contact: Contact communication type (categorical: 'cellular', 'telephone', ...)  
month: Last contact month of year (categorical: 'may', 'nov', ...)  
day_of_week: Last contact day of the week (categorical: 'mon', 'fri', ...)  
duration: Last contact duration, in seconds (numeric). Important note: If duration = 0 then y = 'no'.  

**Campaign information:**

campaign: Number of contacts performed during this campaign and for this client (numeric, includes last contact)  
pdays: Number of days that passed by after the client was last contacted from a previous campaign (numeric)  
previous: Number of contacts performed before this campaign and for this client (numeric)  
poutcome: Outcome of the previous marketing campaign (categorical: 'nonexistent','success', ...)  

**External environment factors:**

emp.var.rate: Employment variation rate - quarterly indicator (numeric)  
cons.price.idx: Consumer price index - monthly indicator (numeric)  
cons.conf.idx: Consumer confidence index - monthly indicator (numeric)  
euribor3m: Euribor 3 month rate - daily indicator (numeric)  
nr.employed: Number of employees - quarterly indicator (numeric)  

**Target variable:**

y: Has the client subscribed a term deposit? (binary: 'yes','no')  


### Data

In [2]:
df1 = pd.read_feather('data/demo_data.feather')
df1.head(5)

,cust_id,age,job,marital,education
0,100,56,housemaid,married,basic.4y
1,101,57,services,married,high.school
2,102,37,services,married,high.school
3,103,40,admin.,married,basic.6y
4,104,56,services,married,high.school


In [3]:
df2 = pd.read_feather('data/campaign_data.feather')
df2.head(5)

,cust_id,campaign_id,campaign,pdays,previous,poutcome
0,100,1010,1,999,0,nonexistent
1,101,1011,1,999,0,nonexistent
2,102,1012,1,999,0,nonexistent
3,103,1013,1,999,0,nonexistent
4,104,1014,1,999,0,nonexistent


In [81]:
# Adding event timestamps to data.

event_timestamp1 = "2020-04-25-00"
event_timestamp2 =  "2020-04-28-00"
df1['event_timestamp']=event_timestamp1
df2['event_timestamp']=event_timestamp2

In [82]:
df1.head(1)

,cust_id,age,job,marital,education,event_timestamp
0,100,56,housemaid,married,basic.4y,2020-04-25-00


## Raw Data Ingestions & Data Transformations

### First Raw Data Ingest/Transformation

In [4]:
namespace="nadeem"

In [5]:
name='demographic_raw'
version='3'
entities=['cust_id']

In [101]:
transform_client = marlin.transform_client(namespace, name,version,entities)

In [102]:
@transform_client.process_function
def process():
    # Write any transformations here...
    transform_client.commit()
    return df1

In [103]:
%%time
process()

CPU times: user 489 ms, sys: 59.8 ms, total: 549 ms
Wall time: 4.3 s


,cust_id,age,job,marital,education,event_timestamp,ingestion_timestamp
0,100,56,housemaid,married,basic.4y,1.587798e+12,1613384619745
1,101,57,services,married,high.school,1.587798e+12,1613384619745
2,102,37,services,married,high.school,1.587798e+12,1613384619745
3,103,40,admin.,married,basic.6y,1.587798e+12,1613384619745
4,104,56,services,married,high.school,1.587798e+12,1613384619745
...,...,...,...,...,...,...,...
41183,41283,73,retired,married,professional.course,1.587798e+12,1613384619745
41184,41284,46,blue-collar,married,professional.course,1.587798e+12,1613384619745
41185,41285,56,retired,married,university.degree,1.587798e+12,1613384619745
41186,41286,44,technician,married,professional.course,1.587798e+12,1613384619745


In [109]:
# Check the Tern UI to see if the data has been loaded with the right version and dependencies.

### Second Raw Data Ingest/Transformation

In [104]:
# Lets define the name, version and entities for the new transform ingest. 
name='campaign_info'
version='1'
entities=['cust_id', 'campaign_id'] 

In [105]:
# Remember we are using the same namespace as the above ingest, 
# we should change if we are writing somewhere else in the Store

transform_client = marlin.transform_client(namespace, name, version, entities)

In [106]:
# we don't have any transformations so the data will be loaded as it is.

@transform_client.process_function
def process():
    # Write any transformations here...
    transform_client.commit()
    return df2

In [107]:
# Execute the process function to register and load the data to the Feature Store.
%%time
process()

CPU times: user 486 ms, sys: 30.5 ms, total: 517 ms
Wall time: 2.14 s


,cust_id,campaign_id,campaign,pdays,previous,poutcome,event_timestamp,ingestion_timestamp
0,100,1010,1,999,0,nonexistent,1.588057e+12,1613384652915
1,101,1011,1,999,0,nonexistent,1.588057e+12,1613384652915
2,102,1012,1,999,0,nonexistent,1.588057e+12,1613384652915
3,103,1013,1,999,0,nonexistent,1.588057e+12,1613384652915
4,104,1014,1,999,0,nonexistent,1.588057e+12,1613384652915
...,...,...,...,...,...,...,...,...
41183,41283,42193,1,999,0,nonexistent,1.588057e+12,1613384652915
41184,41284,42194,1,999,0,nonexistent,1.588057e+12,1613384652915
41185,41285,42195,2,999,0,nonexistent,1.588057e+12,1613384652915
41186,41286,42196,1,999,0,nonexistent,1.588057e+12,1613384652915


In [108]:
# Check the Tern UI to see if the data has been loaded with the right version and dependencies.

### Third Transform: Transformation on data from the feature store

In [134]:
# Lets define the name, version and entities for the new transform ingest. 
name='one_hot_encode_demographic'
version='2'
entities=['cust_id', 'campaign_id'] 

In [135]:



transform_client = marlin.transform_client(namespace, name, version, entities)

In [136]:
# Now we are going to apply transformations to the raw data ingested earlier. We can copy the feature names as a list from the transfrom detail pages (See the UI and select features to copy)

@transform_client.process_function
def process():
    # First step: Define the dependencies requried for data reads
    dep1 = transform_client.add_dependency(namespace, 'demographic_raw', '3',['age','job','marital','education'])    
    demo_data = dep1.read_by_event_date(event_timestamp1, event_timestamp2)
    
    dep2 = transform_client.add_dependency(namespace, 'campaign_info', '1',['previous','campaign','poutcome','pdays'])
    campaign_data = dep2.read_by_event_date(event_timestamp2, event_timestamp2)
    
    # Second step: Read 2 datasets from Feature Store and join them on the cust_id entity
    data = pd.merge(demo_data, campaign_data, how='inner', on=['cust_id'])
    
    # Third step: Apply one hot encoding transfromation to joined data
    transform_output = pd.get_dummies(data) 
    
    # Fouth step: Add the event timestamp
    transform_output['event_timestamp']=  event_timestamp2
    
    transform_client.commit()
    return transform_output

In [137]:
%%time
# Execute the process function to register and load the transformed data to the Feature Store.
# Once completed check on the Tern discovery UI, the updated meta data.

process()

CPU times: user 1.76 s, sys: 241 ms, total: 2.01 s
Wall time: 9.84 s


,age,cust_id,previous,campaign,pdays,campaign_id,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,ingestion_timestamp_x_2021-02-15-02,event_timestamp_x_2020-04-25-00,poutcome_failure,poutcome_nonexistent,poutcome_success,ingestion_timestamp_y_2021-02-15-02,event_timestamp_y_2020-04-28-00,event_timestamp,ingestion_timestamp
0,56,100,0,1,999,1010,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,1,1.588057e+12,1613386567214
1,57,101,0,1,999,1011,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,1,1.588057e+12,1613386567214
2,37,102,0,1,999,1012,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,1,1.588057e+12,1613386567214
3,40,103,0,1,999,1013,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1,1,1.588057e+12,1613386567214
4,56,104,0,1,999,1014,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,1,1.588057e+12,1613386567214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,41283,0,1,999,42193,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,1,1.588057e+12,1613386567214
41184,46,41284,0,1,999,42194,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,1,1.588057e+12,1613386567214
41185,56,41285,0,2,999,42195,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,0,1,0,1,1,1.588057e+12,1613386567214
41186,44,41286,0,1,999,42196,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,1,1.588057e+12,1613386567214


### Fourth Transform: Transformation on data from the feature store

In [146]:
# Lets define the name, version and entities for the new transform ingest. 
name='demographic_job_not_working'
version='2'
entities=['cust_id'] 

In [147]:
transform_client = marlin.transform_client(namespace, name, version, entities)

In [148]:
# Now we are going to apply transformations to the raw data ingested earlier. We can copy the feature names as a list from the transfrom detail pages (See the UI and select features to copy)

@transform_client.process_function
def process():
    # First step: Define the dependencies requried for data transformations
    dep1 = transform_client.add_dependency(namespace, 'demographic_raw', '3',['job'])    
    demo_data = dep1.read_by_event_date(event_timestamp1, event_timestamp2)
   
    # Second step: Creating a new feature from the job data. Inferring someone is "not working" from job status
    demo_data['not_working'] = np.where(np.in1d(demo_data['job'], ['student', 'retired', 'unemployed']), 1, 0) 
    
    # Third step: Add the event timestamp
    demo_data['event_timestamp']=  event_timestamp2
    
    transform_client.commit()
    return demo_data

In [149]:
%%time
# Execute the process function to register and load the transformed data to the Feature Store.
# Once completed check on the Tern discovery UI, the updated meta data.

process()

CPU times: user 999 ms, sys: 78.7 ms, total: 1.08 s
Wall time: 6.68 s


,job,cust_id,ingestion_timestamp,event_timestamp,not_working
0,housemaid,100,1613387482488,1.588057e+12,0
1,services,101,1613387482488,1.588057e+12,0
2,services,102,1613387482488,1.588057e+12,0
3,admin.,103,1613387482488,1.588057e+12,0
4,services,104,1613387482488,1.588057e+12,0
...,...,...,...,...,...
41183,retired,41283,1613387482488,1.588057e+12,1
41184,blue-collar,41284,1613387482488,1.588057e+12,0
41185,retired,41285,1613387482488,1.588057e+12,1
41186,technician,41286,1613387482488,1.588057e+12,0


### Fifth Transform: Transformation on data from the feature store

In [150]:
# Lets define the name, version and entities for the new transform ingest. 
name='campaign_no_previous_contact'
version='1'
entities=['cust_id','campaign_id'] 

In [151]:
transform_client = marlin.transform_client(namespace, name, version, entities)

In [152]:
# Now we are going to apply transformations to the raw data ingested earlier. We can copy the feature names as a list from the transfrom detail pages (See the UI and select features to copy)

@transform_client.process_function
def process():
    # First step: Define the dependencies requried for data transformations
    dep1 = transform_client.add_dependency(namespace, 'campaign_info', '1',['pdays'])
    campaign_data = dep1.read_by_event_date(event_timestamp2, event_timestamp2)
   
    # Second step: Creating a new feature from the job data. Inferring someone is "not working" from job status
    campaign_data['no_previous_contact'] = np.where(campaign_data['pdays'] == 999, 1, 0)   
    
    # Third step: Add the event timestamp
    campaign_data['event_timestamp']=  event_timestamp2
    
    transform_client.commit()
    return campaign_data

In [153]:
%%time
# Execute the process function to register and load the transformed data to the Feature Store.
# Once completed check on the Tern discovery UI, the updated meta data.

process()

CPU times: user 1.07 s, sys: 82.3 ms, total: 1.15 s
Wall time: 9.36 s


,pdays,campaign_id,cust_id,ingestion_timestamp,event_timestamp,no_previous_contact
0,999,1010,100,1613387572710,1.588057e+12,1
1,999,1011,101,1613387572710,1.588057e+12,1
2,999,1012,102,1613387572710,1.588057e+12,1
3,999,1013,103,1613387572710,1.588057e+12,1
4,999,1014,104,1613387572710,1.588057e+12,1
...,...,...,...,...,...,...
41183,999,42193,41283,1613387572710,1.588057e+12,1
41184,999,42194,41284,1613387572710,1.588057e+12,1
41185,999,42195,41285,1613387572710,1.588057e+12,1
41186,999,42196,41286,1613387572710,1.588057e+12,1


### Exploration Client

In [52]:
exploration_client=marlin.exploration_client()
df=exploration_client.get_transform(namespace, 'one_hot_encode_demographic','1')

In [53]:
data=df.read_by_event_date("2020-04-15","2020-05-01", "%Y-%m-%d")

In [54]:
data

,marital_single,marital_unknown,job_retired,job_technician,age,education_basic.6y,job_entrepreneur,ingestion_timestamp_2021-02-12-05,job_self-employed,education_professional.course,marital_married,job_student,job_services,marital_divorced,education_high.school,job_admin.,job_unknown,education_basic.9y,job_management,education_illiterate,education_university.degree,education_basic.4y,education_unknown,job_blue-collar,job_housemaid,event_timestamp_2020-04-25-00,job_unemployed,cust_id,ingestion_timestamp,event_timestamp
0,0,0,0,0,56,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,100,2021-02-12-05,2020-04-30-00
1,0,0,0,0,57,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,101,2021-02-12-05,2020-04-30-00
2,0,0,0,0,37,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,102,2021-02-12-05,2020-04-30-00
3,0,0,0,0,40,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,103,2021-02-12-05,2020-04-30-00
4,0,0,0,0,56,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,104,2021-02-12-05,2020-04-30-00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,0,0,1,0,73,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,41283,2021-02-12-05,2020-04-30-00
41184,0,0,0,0,46,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,41284,2021-02-12-05,2020-04-30-00
41185,0,0,1,0,56,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,41285,2021-02-12-05,2020-04-30-00
41186,0,0,0,1,44,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,41286,2021-02-12-05,2020-04-30-00
